In [2]:
using DataFrames, CSV, XLSX
using Optim
using PyPlot

┌ Info: Precompiling XLSX [fdbf4ff8-1666-58a4-91e7-1b58723a45e0]
└ @ Base loading.jl:1260


# Read data from xlsx

In [3]:
fname = "Ausw17.xlsx"

f = XLSX.readxlsx(fname)

XLSXFile("Ausw17.xlsx") containing 1 Worksheet
            sheetname size          range        
-------------------------------------------------
        Auswertung 17 79x8          A1:H79       


In [4]:
car1 = DataFrame(XLSX.readtable(fname, "Auswertung 17", "A:B", 
        column_labels = ["time", "pos"],
        infer_eltypes = true
            )...)
first(car1, 4)

,time,pos
,Any,Float64
1,0,0.858483
2,0.0333333,0.85912
3,0.0666667,0.860149
4,0.1,0.861316


In [5]:
car1.time = float.(car1.time)
first(car1, 4)

,time,pos
,Float64,Float64
1,0.0,0.858483
2,0.0333333,0.85912
3,0.0666667,0.860149
4,0.1,0.861316


In [ ]:
car2 = DataFrame(XLSX.readtable(fname, "Auswertung 17", "F:G", 
        column_labels = ["time", "pos"],
        infer_eltypes = true
            )...)
first(car2, 4)

In [ ]:
car2.time = float.(car2.time)
first(car2, 4)

In [ ]:
scatter(car1.time, car1.pos, label = "car 1")
scatter(car2.time, car2.pos, label = "car 2")
legend()

# Modelling

In [ ]:
function two_piece_linear(x, para)
    n   = para[1]
    m1  = para[2]
    m2  = para[3]
    x12 = para[4]
    if x < x12
        n + m1 * x
    else
        n + m1 * x12 + m2 * (x - x12)
    end
end

In [ ]:
scatter(car1.time, car1.pos, label = "car 1")
scatter(car2.time, car2.pos, label = "car 2")


# car 1
times = 0:0.01:2.5
para1 = [0.85, 0.06, 0.3, 1.1]
model_car1 = map(t -> two_piece_linear(t, para1), times)
plot(times, model_car1, color = "lightblue", label = "model 1")

# car 2
para2 = [0.0, 0.4, 0.1, 1.2]
model_car2 = map(t -> two_piece_linear(t, para2), times)
plot(times, model_car2, color = "red", label = "model 2")

legend()

## Parameter fitting

In [ ]:
function sum_residuals_squared(observed_data, model_parameter)
    t = observed_data[1]
    x = observed_data[2]
    
    model_x = map(t -> two_piece_linear(t, model_parameter), t)
    
    sum((x .- model_x) .^ 2)
end

In [ ]:
observed_data = [car1.time, car1.pos]
likelihood(model_parameter) = sum_residuals_squared(observed_data, model_parameter)
@show res = optimize(likelihood, para1)
para1 = Optim.minimizer(res)

In [ ]:
observed_data = [car2.time, car2.pos]
likelihood(model_parameter) = sum_residuals_squared(observed_data, model_parameter)
@show res = optimize(likelihood, para2)
para2 = Optim.minimizer(res)

In [ ]:
scatter(car1.time, car1.pos, label = "car 1")
scatter(car2.time, car2.pos, label = "car 2")


# car 1
times = 0:0.01:2.5
model_car1 = map(t -> two_piece_linear(t, para1), times)
plot(times, model_car1, color = "lightblue", label = "model 1")

# car 2
model_car2 = map(t -> two_piece_linear(t, para2), times)
plot(times, model_car2, color = "red", label = "model 2")
legend()

In [ ]:
@show x12_1 = para1[4] 
@show x12_2 = para2[4]
x12_1 -x12_2

In [ ]:
m1 = 1; m2 = 1
@show p_before = m1 * para1[2] + m2 * para2[2]
@show p_after  = m1 * para1[3] + m2 * para2[3];

In [ ]:
(p_before - p_after) / ((p_before + p_after) / 2)

In [ ]:
@show E_before = m1 * para1[2]^2 / 2 + m2 * para2[2]^2 / 2
@show E_after  = m1 * para1[3]^2 / 2 + m2 * para2[3]^2 / 2

In [ ]:
filter!(row -> row.time < 1.0 || row.time > 1.5, car1 );
filter!(row -> row.time < 1.0 || row.time > 1.5, car2 );

In [1]:
car1

UndefVarError: UndefVarError: car1 not defined